In [3]:
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, random_split, DataLoader
from PIL import Image
import torchvision.models as models
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
import torch.nn.functional as F
from sklearn import tree
import matplotlib.pyplot as plt

In [2]:
# replace the path 
path = "C:\\Users\\czahi\\Desktop\\CIS519-Final-Project"

# Train the CNN Model

In [14]:
class CUBDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.df = pd.read_csv(csv_file)

        # # for test, only first 200 samples
        # self.df = self.df.head(1000)

        self.transform = transform
        self.root_dir = root_dir
        
    def __len__(self):
        return len(self.df)    
    
    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img_path, img_label = row['image_path'], torch.tensor(row[1:].astype('float32').values)
        img_path = os.path.join(self.root_dir, img_path)
        
        # for Windows path
        img_path = img_path.replace("/", "\\")
        img = Image.open(img_path)
        if self.transform:
            img = self.transform(img)
        return img, img_label

In [15]:
def F_score(output, label, threshold=0.5, beta=1):
    prob = output > threshold
    label = label > threshold

    TP = (prob & label).sum(1).float()
    TN = ((~prob) & (~label)).sum(1).float()
    FP = (prob & (~label)).sum(1).float()
    FN = ((~prob) & label).sum(1).float()

    precision = torch.mean(TP / (TP + FP + 1e-12))
    recall = torch.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return F2.mean(0)

In [16]:
class MultilabelImageClassificationBase(torch.nn.Module):
    def training_step(self, batch):
        images, targets = batch 
        out = self(images)                      
        loss = F.binary_cross_entropy(out, targets)      
        return loss
    
    def validation_step(self, batch):
        images, targets = batch 
        out = self(images)                           # Generate predictions
        loss = F.binary_cross_entropy(out, targets)  # Calculate loss
        score = F_score(out, targets)
        return {'val_loss': loss.detach(), 'val_score': score.detach() }
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_scores = [x['val_score'] for x in outputs]
        epoch_score = torch.stack(batch_scores).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_score': epoch_score.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_score: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_score']))

In [17]:
class CNN(MultilabelImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = models.wide_resnet101_2(pretrained=True)
        # Replace last layer
        num_ftrs = self.network.fc.in_features
#         self.network.fc = torch.nn.Sequential(
#             torch.nn.Linear(num_ftrs, 288),
#             torch.nn.Softmax())
        self.network.fc = torch.nn.Linear(num_ftrs, 288)
    
    def forward(self, xb):
        return torch.sigmoid(self.network(xb))

In [18]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [19]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [20]:
# load dataset
transform = transforms.Compose([transforms.ToTensor()])
dataset = CUBDataset(os.path.join(path, "data\\csv\\seen_images.csv"), path, transform=transform)
len(dataset)

5263

In [21]:
# split dataset into train and validation datasets
val_pct = 0.15
val_size = int(val_pct * len(dataset))
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

In [22]:
batch_size = 16
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=0, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=0, pin_memory=True)

In [23]:
# Run on GPU
device = get_default_device()
device

device(type='cuda')

In [24]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)

In [25]:
model = to_device(CNN(), device)

In [26]:
num_epochs = 20
opt_func = torch.optim.Adam
lr = 1e-5

In [77]:
# evaluate before training
evaluate(model, val_dl)

{'val_loss': 0.7081231474876404, 'val_score': 0.5505195260047913}

In [78]:
# train
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)


Epoch [0], train_loss: 0.5430, val_loss: 0.4626, val_score: 0.8217



Epoch [1], train_loss: 0.4202, val_loss: 0.3975, val_score: 0.8438



Epoch [2], train_loss: 0.3808, val_loss: 0.3736, val_score: 0.8556



Epoch [3], train_loss: 0.3573, val_loss: 0.3586, val_score: 0.8622



Epoch [4], train_loss: 0.3380, val_loss: 0.3467, val_score: 0.8663


...


Epoch [15], train_loss: 0.2234, val_loss: 0.2891, val_score: 0.8890



Epoch [16], train_loss: 0.2131, val_loss: 0.2842, val_score: 0.8895



Epoch [17], train_loss: 0.2041, val_loss: 0.2795, val_score: 0.8925



Epoch [18], train_loss: 0.1946, val_loss: 0.2758, val_score: 0.8950



Epoch [19], train_loss: 0.1853, val_loss: 0.2749, val_score: 0.8936


In [36]:
# save model
torch.save(model.state_dict(), "WideResNet_20epoch.pt")

In [28]:
# load model 
device = torch.device("cuda")
model2 = CNN()
model2.load_state_dict(torch.load("WideResNet50_20epoch.pt"))
model2.eval()
model2.to(device)

CNN(
  (network): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
        

In [30]:
evaluate(model2, val_dl)

{'val_loss': 0.1429191678762436, 'val_score': 0.9530104398727417}

In [31]:
model = model2

In [32]:
bird_attr_df = pd.read_csv(os.path.join(path, "data\\csv\\bird_to_attrs.csv"))

In [33]:
bird_attr_df.head()

,bird_name,has_crown_color::blue,has_crown_color::black,has_crown_color::orange,has_crown_color::buff,has_crown_color::brown,has_crown_color::grey,has_crown_color::white,has_crown_color::red,has_crown_color::pink,...,has_wing_color::rufous,has_wing_color::orange,has_wing_color::red,has_wing_color::green,has_wing_color::pink,has_belly_color::purple,has_eye_color::purple,has_underparts_color::purple,has_under_tail_color::pink,has_eye_color::pink
0,Downy Woodpecker,1,1,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,Song Sparrow,0,1,1,1,1,1,1,1,0,...,1,1,0,0,0,0,0,0,0,0
2,Northern Flicker,1,1,1,1,1,1,1,1,0,...,0,0,0,1,0,0,0,0,0,0
3,Vesper Sparrow,0,1,0,1,1,1,1,0,0,...,0,1,0,0,1,1,0,0,0,0
4,Philadelphia Vireo,0,1,0,1,1,1,1,0,0,...,0,0,0,1,0,0,0,0,0,0


In [34]:
temp = bird_attr_df.to_numpy()
train_x = np.copy(temp[:, 1:]).astype(np.int64)
train_y = temp[:, 0]
train_y.reshape(-1, 1)
print(train_y.shape)

(200,)


In [35]:
train_y

array(['Downy Woodpecker', 'Song Sparrow', 'Northern Flicker',
       'Vesper Sparrow', 'Philadelphia Vireo', 'Bohemian Waxwing',
       'Caspian Tern', 'Pileated Woodpecker', 'Western Grebe',
       'Pied Kingfisher', 'Cape Glossy Starling',
       'Scissor tailed Flycatcher', 'Red bellied Woodpecker', 'Green Jay',
       'Eared Grebe', 'Black billed Cuckoo', 'White breasted Kingfisher',
       'Black throated Sparrow', 'Canada Warbler', 'Clark Nutcracker',
       'Prairie Warbler', 'Pacific Loon', 'Yellow billed Cuckoo',
       'Black footed Albatross', 'Loggerhead Shrike', 'Hooded Oriole',
       'Northern Waterthrush', 'Tropical Kingbird', 'Acadian Flycatcher',
       'Winter Wren', 'White throated Sparrow', 'Black and white Warbler',
       'Yellow breasted Chat', 'White breasted Nuthatch',
       'Cape May Warbler', 'Gadwall', 'Wilson Warbler', 'Blue Jay',
       'Gray Kingbird', 'Glaucous winged Gull', 'Blue winged Warbler',
       'Prothonotary Warbler', 'Warbling Vireo', 'Brow

In [36]:
clf = tree.DecisionTreeClassifier(max_depth=200)
clf.fit(train_x, train_y)

DecisionTreeClassifier(max_depth=200)

# Load Test Dataset

In [37]:
transform = transforms.Compose([transforms.ToTensor()])
# test dataset
test_dataset = CUBDataset(os.path.join(path, "data\\csv\\test_images.csv"), path, transform=transform)
test_df = pd.read_csv(os.path.join(path, "data\\csv\\test_images.csv"))

In [38]:
print(len(test_dataset))

500


In [39]:
seen_image_path_df = pd.read_csv(os.path.join(path, "data\\csv\\seen_images.csv"))

In [42]:
seen_image_to_bird = []
for i in range(len(seen_image_path_df)):
    image_path = seen_image_path_df.loc[i][0]
    print(image_path)
    folder_name = image_path.split("\\")[2]
    bird_name = folder_name.split(".")[1].replace("_", " ")
    print(bird_name)
    seen_image_to_bird.append(bird_name)

data\seen_images\001.Black_footed_Albatross\Black_footed_Albatross_0001_2950163169.jpg
Black footed Albatross
data\seen_images\001.Black_footed_Albatross\Black_footed_Albatross_0002_2293084168.jpg
Black footed Albatross
data\seen_images\001.Black_footed_Albatross\Black_footed_Albatross_0003_2981373810.jpg
Black footed Albatross
data\seen_images\001.Black_footed_Albatross\Black_footed_Albatross_0004_2731401028.jpg
Black footed Albatross
data\seen_images\001.Black_footed_Albatross\Black_footed_Albatross_0005_2755588934.jpg
Black footed Albatross
data\seen_images\001.Black_footed_Albatross\Black_footed_Albatross_0006_2928372528.jpg
Black footed Albatross
data\seen_images\001.Black_footed_Albatross\Black_footed_Albatross_0007_2675126617.jpg
Black footed Albatross
data\seen_images\001.Black_footed_Albatross\Black_footed_Albatross_0008_1384283201.jpg
Black footed Albatross
data\seen_images\001.Black_footed_Albatross\Black_footed_Albatross_0009_2408326989.jpg
Black footed Albatross
data\seen_

data\seen_images\029.American_Crow\American_Crow_0013_3051549869.jpg
American Crow
data\seen_images\029.American_Crow\American_Crow_0014_2769157461.jpg
American Crow
data\seen_images\029.American_Crow\American_Crow_0015_2365091282.jpg
American Crow
data\seen_images\029.American_Crow\American_Crow_0016_2753381796.jpg
American Crow
data\seen_images\029.American_Crow\American_Crow_0017_2359869348.jpg
American Crow
data\seen_images\029.American_Crow\American_Crow_0018_1877680209.jpg
American Crow
data\seen_images\029.American_Crow\American_Crow_0019_2863695170.jpg
American Crow
data\seen_images\029.American_Crow\American_Crow_0020_2934449411.jpg
American Crow
data\seen_images\029.American_Crow\American_Crow_0021_2626777569.jpg
American Crow
data\seen_images\029.American_Crow\American_Crow_0022_2975254436.jpg
American Crow
data\seen_images\029.American_Crow\American_Crow_0023_xxx.jpg
American Crow
data\seen_images\029.American_Crow\American_Crow_0024_2618947526.jpg
American Crow
data\seen_i

data\seen_images\052.Pied_billed_Grebe\Pied_billed_Grebe_0022_397190380.jpg
Pied billed Grebe
data\seen_images\052.Pied_billed_Grebe\Pied_billed_Grebe_0023_674336278.jpg
Pied billed Grebe
data\seen_images\052.Pied_billed_Grebe\Pied_billed_Grebe_0024_389578482.jpg
Pied billed Grebe
data\seen_images\052.Pied_billed_Grebe\Pied_billed_Grebe_0025_673188429.jpg
Pied billed Grebe
data\seen_images\052.Pied_billed_Grebe\Pied_billed_Grebe_0026_2159703847.jpg
Pied billed Grebe
data\seen_images\052.Pied_billed_Grebe\Pied_billed_Grebe_0027_2531330942.jpg
Pied billed Grebe
data\seen_images\052.Pied_billed_Grebe\Pied_billed_Grebe_0028_674298766.jpg
Pied billed Grebe
data\seen_images\052.Pied_billed_Grebe\Pied_billed_Grebe_0029_300538392.jpg
Pied billed Grebe
data\seen_images\052.Pied_billed_Grebe\Pied_billed_Grebe_0030_2503376738.jpg
Pied billed Grebe
data\seen_images\053.Western_Grebe\Western_Grebe_0001_477238421.jpg
Western Grebe
data\seen_images\053.Western_Grebe\Western_Grebe_0002_482019868.jpg
W

data\seen_images\074.Florida_Jay\Florida_Jay_0023_739264851.jpg
Florida Jay
data\seen_images\074.Florida_Jay\Florida_Jay_0024_2300511239.jpg
Florida Jay
data\seen_images\074.Florida_Jay\Florida_Jay_0025_2896321030.jpg
Florida Jay
data\seen_images\074.Florida_Jay\Florida_Jay_0026_2267626839.jpg
Florida Jay
data\seen_images\074.Florida_Jay\Florida_Jay_0027_2327395056.jpg
Florida Jay
data\seen_images\074.Florida_Jay\Florida_Jay_0028_2485111000.jpg
Florida Jay
data\seen_images\074.Florida_Jay\Florida_Jay_0029_2721947456.jpg
Florida Jay
data\seen_images\074.Florida_Jay\Florida_Jay_0030_458297926.jpg
Florida Jay
data\seen_images\074.Florida_Jay\Florida_Jay_0031_266193041.jpg
Florida Jay
data\seen_images\074.Florida_Jay\Florida_Jay_0032_2565468217.jpg
Florida Jay
data\seen_images\074.Florida_Jay\Florida_Jay_0033_1233902148.jpg
Florida Jay
data\seen_images\074.Florida_Jay\Florida_Jay_0034_2565468527.jpg
Florida Jay
data\seen_images\074.Florida_Jay\Florida_Jay_0035_171539494.jpg
Florida Jay
dat

data\seen_images\097.Orchard_Oriole\Orchard_Oriole_0022_549995638.jpg
Orchard Oriole
data\seen_images\097.Orchard_Oriole\Orchard_Oriole_0023_176074503.jpg
Orchard Oriole
data\seen_images\097.Orchard_Oriole\Orchard_Oriole_0024_2599230832.jpg
Orchard Oriole
data\seen_images\097.Orchard_Oriole\Orchard_Oriole_0025_450468470.jpg
Orchard Oriole
data\seen_images\097.Orchard_Oriole\Orchard_Oriole_0026_137280806.jpg
Orchard Oriole
data\seen_images\097.Orchard_Oriole\Orchard_Oriole_0027_2678724108.jpg
Orchard Oriole
data\seen_images\097.Orchard_Oriole\Orchard_Oriole_0028_2481603085.jpg
Orchard Oriole
data\seen_images\097.Orchard_Oriole\Orchard_Oriole_0029_338933477.jpg
Orchard Oriole
data\seen_images\097.Orchard_Oriole\Orchard_Oriole_0030_1420273185.jpg
Orchard Oriole
data\seen_images\097.Orchard_Oriole\Orchard_Oriole_0031_2432980950.jpg
Orchard Oriole
data\seen_images\097.Orchard_Oriole\Orchard_Oriole_0032_2561716164.jpg
Orchard Oriole
data\seen_images\097.Orchard_Oriole\Orchard_Oriole_0033_248

data\seen_images\121.Grasshopper_Sparrow\Grasshopper_Sparrow_0020_758169484.jpg
Grasshopper Sparrow
data\seen_images\121.Grasshopper_Sparrow\Grasshopper_Sparrow_0021_2520481516.jpg
Grasshopper Sparrow
data\seen_images\121.Grasshopper_Sparrow\Grasshopper_Sparrow_0022_2862896572.jpg
Grasshopper Sparrow
data\seen_images\121.Grasshopper_Sparrow\Grasshopper_Sparrow_0023_2549019821.jpg
Grasshopper Sparrow
data\seen_images\121.Grasshopper_Sparrow\Grasshopper_Sparrow_0024_565943618.jpg
Grasshopper Sparrow
data\seen_images\121.Grasshopper_Sparrow\Grasshopper_Sparrow_0025_716973078.jpg
Grasshopper Sparrow
data\seen_images\121.Grasshopper_Sparrow\Grasshopper_Sparrow_0026_1233911501.jpg
Grasshopper Sparrow
data\seen_images\121.Grasshopper_Sparrow\Grasshopper_Sparrow_0027_xxx.jpg
Grasshopper Sparrow
data\seen_images\121.Grasshopper_Sparrow\Grasshopper_Sparrow_0028_653097717.jpg
Grasshopper Sparrow
data\seen_images\121.Grasshopper_Sparrow\Grasshopper_Sparrow_0029_533476995.jpg
Grasshopper Sparrow
da

data\seen_images\146.Forsters_Tern\Forsters_Tern_0007_308527199.jpg
Forsters Tern
data\seen_images\146.Forsters_Tern\Forsters_Tern_0008_1501120971.jpg
Forsters Tern
data\seen_images\146.Forsters_Tern\Forsters_Tern_0009_2779665089.jpg
Forsters Tern
data\seen_images\146.Forsters_Tern\Forsters_Tern_0010_1604267510.jpg
Forsters Tern
data\seen_images\146.Forsters_Tern\Forsters_Tern_0011_2234274249.jpg
Forsters Tern
data\seen_images\146.Forsters_Tern\Forsters_Tern_0012_2208266536.jpg
Forsters Tern
data\seen_images\146.Forsters_Tern\Forsters_Tern_0013_2083114719.jpg
Forsters Tern
data\seen_images\146.Forsters_Tern\Forsters_Tern_0014_2803159992.jpg
Forsters Tern
data\seen_images\146.Forsters_Tern\Forsters_Tern_0015_2857554307.jpg
Forsters Tern
data\seen_images\146.Forsters_Tern\Forsters_Tern_0016_2718730358.jpg
Forsters Tern
data\seen_images\146.Forsters_Tern\Forsters_Tern_0017_1013968744.jpg
Forsters Tern
data\seen_images\146.Forsters_Tern\Forsters_Tern_0018_1288270890.jpg
Forsters Tern
data\

data\seen_images\169.Magnolia_Warbler\Magnolia_Warbler_0016_2542370241.jpg
Magnolia Warbler
data\seen_images\169.Magnolia_Warbler\Magnolia_Warbler_0017_2519116831.jpg
Magnolia Warbler
data\seen_images\169.Magnolia_Warbler\Magnolia_Warbler_0018_1394644922.jpg
Magnolia Warbler
data\seen_images\169.Magnolia_Warbler\Magnolia_Warbler_0019_2524515128.jpg
Magnolia Warbler
data\seen_images\169.Magnolia_Warbler\Magnolia_Warbler_0020_2501274595.jpg
Magnolia Warbler
data\seen_images\169.Magnolia_Warbler\Magnolia_Warbler_0021_2455511212.jpg
Magnolia Warbler
data\seen_images\169.Magnolia_Warbler\Magnolia_Warbler_0022_xxx.jpg
Magnolia Warbler
data\seen_images\169.Magnolia_Warbler\Magnolia_Warbler_0023_125338593.jpg
Magnolia Warbler
data\seen_images\169.Magnolia_Warbler\Magnolia_Warbler_0024_496559340.jpg
Magnolia Warbler
data\seen_images\169.Magnolia_Warbler\Magnolia_Warbler_0025_490669701.jpg
Magnolia Warbler
data\seen_images\170.Mourning_Warbler\Mourning_Warbler_0001_2099292330.jpg
Mourning Warble

data\seen_images\191.Red_headed_Woodpecker\Red_headed_Woodpecker_0016_410463447.jpg
Red headed Woodpecker
data\seen_images\191.Red_headed_Woodpecker\Red_headed_Woodpecker_0017_2917925915.jpg
Red headed Woodpecker
data\seen_images\191.Red_headed_Woodpecker\Red_headed_Woodpecker_0018_2579398144.jpg
Red headed Woodpecker
data\seen_images\191.Red_headed_Woodpecker\Red_headed_Woodpecker_0019_454761427.jpg
Red headed Woodpecker
data\seen_images\191.Red_headed_Woodpecker\Red_headed_Woodpecker_0020_2599542610.jpg
Red headed Woodpecker
data\seen_images\191.Red_headed_Woodpecker\Red_headed_Woodpecker_0021_566993404.jpg
Red headed Woodpecker
data\seen_images\191.Red_headed_Woodpecker\Red_headed_Woodpecker_0022_2720313574.jpg
Red headed Woodpecker
data\seen_images\191.Red_headed_Woodpecker\Red_headed_Woodpecker_0023_2714536014.jpg
Red headed Woodpecker
data\seen_images\191.Red_headed_Woodpecker\Red_headed_Woodpecker_0024_316230604.jpg
Red headed Woodpecker
data\seen_images\191.Red_headed_Woodpecke

In [43]:
len(seen_image_to_bird)

5263

In [44]:
image_to_bird = []
for i in range(len(test_df)):
    image_path = test_df.loc[i][0]
    folder_name = image_path.split("\\")[2]
    bird_name = folder_name.split(".")[1].replace("_", " ")
    print(bird_name)
    image_to_bird.append(bird_name)

Dark eyed Junco
Red winged Blackbird
Tropical Kingbird
Pelagic Cormorant
Painted Bunting
Blue Grosbeak
Least Tern
Heermann Gull
House Sparrow
Baird Sparrow
Gadwall
Black footed Albatross
Barn Swallow
Western Wood Pewee
Ovenbird
Rock Wren
Red eyed Vireo
Cardinal
Green tailed Towhee
Canada Warbler
Great Crested Flycatcher
Mourning Warbler
House Wren
Pine Grosbeak
American Redstart
Glaucous winged Gull
Northern Flicker
Boat tailed Grackle
Magnolia Warbler
Elegant Tern
American Pipit
Brewer Sparrow
Red cockaded Woodpecker
Bank Swallow
Prothonotary Warbler
Great Crested Flycatcher
Seaside Sparrow
Green Kingfisher
Green tailed Towhee
Nighthawk
Nighthawk
Yellow billed Cuckoo
Herring Gull
American Goldfinch
House Wren
Golden winged Warbler
Gadwall
Blue headed Vireo
Rusty Blackbird
Red eyed Vireo
Winter Wren
Clark Nutcracker
Barn Swallow
Olive sided Flycatcher
Pied Kingfisher
Pacific Loon
White Pelican
Eastern Towhee
White breasted Kingfisher
Yellow throated Vireo
Brown Pelican
Golden winged Wa

In [45]:
len(image_to_bird)

500

# Predict

In [10]:
def read_bird(filename="bird_to_attrs.csv"):
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        i = 0
        bird_name = []
        attr_list = []
        for row in reader:
            if i == 0:
                attribute = row[1:]
            else:
                bird_name.append(row[0])
                attr_list.append(row[1:])
            i += 1
    bird_name = np.array(bird_name)
    attribute = np.array(attribute)
    attr_list = np.array(attr_list).astype(np.int)
    return bird_name, attribute, attr_list

In [11]:
def log_weight_attr(attr_list, k):
    attr_count = np.zeros(len(attr_list[0]))
    for i in range(len(attr_list[0])):
        attr_count[i] = (np.sum(attr_list[:,i]))
    
    nor_attr_count = attr_count / np.max(attr_count)
    nor_attr_count2 = nor_attr_count
    for i in range(288):
        if(nor_attr_count2[i] == 0):
            nor_attr_count2[i] = 1
    nor_attr_count3 = (-1*k)* np.log(nor_attr_count2)
    attr_list_log = attr_list * nor_attr_count3
    return attr_list_log

In [12]:
def bird_index_lookup(bird_list, name):
    index = 0
    for bird in bird_list:
        if name == bird:
            bird_index = index
            break
        index += 1
    return bird_index

In [13]:
def accuracy(perdict, Y):
    count = np.sum(np.equal(perdict, Y))
    return len(predict), count, count / len(Y)

In [14]:
def load_CNN(filename="cnn_res.csv"):
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        i = 0
        predict_x = []
        bird_y = []
        for row in reader:
            word = (row[0])
            if i != 0:
                predict_x.append(np.array(row[1:]).astype(np.int))
                path = word.split('/')
                bird = path[2].split('.')[1]
                name = bird.replace('_', ' ')
                bird_y.append(bird_index_lookup(bird_name, name))
            i += 1
    predict_x = np.array(predict_x)
    bird_y = np.array(bird_y)
    
    return predict_x, bird_y

In [15]:
def make_prediction(attr_list_log2, predict_x):
    result = np.zeros(predict_x.shape[0])
    for i in range(predict_x.shape[0]):
        score = np.zeros(200)
        for j in range(200):
            score[j] = np.dot(predict_x[i], attr_list_log2[j])
        result[i] = np.argmax(score)
    return result

In [46]:
device = torch.device("cuda")
model = CNN()
model.load_state_dict(torch.load("WideResNet50_20epoch.pt"))
model.eval()
model.to(device)

CNN(
  (network): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
        

In [47]:
def predict_single(model, image):
    xb = image.unsqueeze(0)
    xb = to_device(xb, device)
    preds = model(xb)
    prediction = preds[0]
    return prediction

In [48]:
def decode_pred(pred, threshold=0.5):
    pred = pred.cpu().detach().numpy()
    pred = np.where(pred > threshold, 1, 0)
    return pred

In [49]:
# CNN result of a single sample
pred = predict_single(model, dataset[0][0])
pred = decode_pred(pred)
truth = dataset[0][1].numpy()
print(pred == truth)
pred = pred.reshape((1, 288))
res = clf.predict(pred)
print(res)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [50]:
pred = predict_single(model, test_dataset[0][0])
print(pred)
pred = decode_pred(pred, threshold=0.25)
truth = test_dataset[0][1].numpy()
print((pred == truth).sum())
pred = pred.reshape((1, 288))
res = clf.predict(pred)
res[0]

tensor([8.4150e-01, 9.9976e-01, 1.4466e-02, 9.5583e-01, 9.7665e-01, 9.9927e-01,
        2.1307e-01, 1.5010e-03, 1.5375e-02, 5.8114e-02, 8.5457e-01, 7.7457e-02,
        2.6971e-01, 1.0160e-02, 2.3705e-03, 9.9621e-01, 9.9969e-01, 9.8352e-01,
        9.9654e-01, 9.9984e-01, 4.3688e-02, 6.2279e-03, 2.9943e-02, 8.7196e-01,
        9.4023e-02, 2.7979e-02, 8.9414e-01, 3.5841e-02, 9.0174e-04, 2.0896e-02,
        9.9920e-01, 9.9909e-01, 5.1305e-01, 6.5085e-02, 9.7362e-02, 3.3101e-01,
        5.7287e-02, 8.3930e-03, 6.3024e-02, 9.5765e-01, 3.0684e-01, 9.8953e-01,
        9.9964e-01, 1.8522e-01, 9.4698e-01, 1.5801e-01, 3.1204e-01, 9.0980e-01,
        9.9999e-01, 9.8797e-01, 9.9959e-01, 9.2796e-01, 2.4289e-01, 9.9234e-01,
        9.9698e-01, 9.7836e-01, 9.9053e-01, 9.9672e-01, 5.6408e-02, 9.9571e-01,
        4.4459e-03, 8.4407e-01, 1.3037e-01, 4.2698e-02, 3.3075e-01, 1.5640e-02,
        5.8362e-01, 4.8909e-04, 9.9990e-01, 8.6780e-01, 3.6071e-01, 9.9921e-01,
        9.7532e-01, 9.9925e-01, 3.6440e-

'Black billed Cuckoo'

In [54]:
def compute_distance(pred, bird_attr):
    res = 0
    for i in range(len([pred])):
        if (pred[i] == bird_attr[i] and pred[i] == 1):
            res -= 100
        elif (pred[i] == bird_attr[i] and pred[i] == 0):
            res -= 1
        else:
            res += 1
    return res

In [55]:
def get_pred_bird(pred, bird_attr_df):
    
    res = ""
    min_d = 300
    for i in range(len(bird_attr_df)):
        temp = bird_attr_df.iloc[i].to_numpy()
        bird_name = temp[0]
        bird_attr = temp[1:].astype(np.int64)
        d = compute_distance(pred, bird_attr)
        if d < min_d:
            min_d = d
            res = bird_name
    
    return res

In [51]:
def predict_all(test_dataset, image_to_bird, clf, threshold=0.5):
    total = 0
    num_correct = 0
    for i in range(len(test_dataset)):
        img = test_dataset[i][0]
        attrs = test_dataset[i][1]
        pred = predict_single(model, img)
#         pred = decode_pred(pred, threshold)
        pred = pred.cpu().detach().numpy()
        pred = pred.reshape((1, 288))
        pred = clf.predict(pred)[0]
        if (pred == image_to_bird[total]):
            num_correct += 1
        total += 1
    return total, num_correct, num_correct/total

In [1]:
def predict_all_distance(test_dataset, bird_attr_df, image_to_bird, threshold=0.5):
    total = 0
    num_correct = 0
    for i in range(len(test_dataset)):
        img = test_dataset[i][0]
        pred = predict_single(model, img)
        pred = pred.cpu().detach().numpy()
        pred = get_pred_bird(pred, bird_attr_df)
        if (pred == image_to_bird[total]):
            num_correct += 1
        total += 1
    return total, num_correct, num_correct/total

In [52]:
# test dataset 
attr_list_log2 = log_weight_attr(attr_list, k)
predict_x, bird_y = load_CNN()
pre_y = make_prediction(attr_list_log2, predict_x)
print(accuracy(result, bird_y))

(500, 156, 0.312)


## Generate CNN prediction result as csv

In [44]:
def generate_cnn_result(test_df, test_dataset):
    res = test_df[0:0]
    total = 0
    for i in range(len(test_dataset)):
        img = test_dataset[i][0]
        pred = predict_single(model, img)
        pred = decode_pred(pred, threshold=0.5)
        res.loc[total] = [test_df.iloc[total]["image_path"]] + list(pred)
        total += 1
    return res

In [45]:
cnn_res_df = generate_cnn_result(test_df, test_dataset)

C:\Users\czahi\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [50]:
cnn_res_df.to_csv("cnn_res.csv", index=False)